# Grover's Algorithm using 8 qubits

Step 1: Define the Oracle. I use a random number generator to create the oracle so I never know who the winners actually are. First we make the classical Oracle that returns True if winner is detected. Then we define the unitary matrix used that acts as the quantum oracle. It will flip the phase of only the winners. Imagine this oracle being defined once in a server.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.circuit.library import UnitaryGate
from qiskit_aer.primitives import Sampler
import random
from math import sqrt,ceil,floor
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
from qiskit.result import QuasiDistribution

aer_sim = AerSimulator()
sampler = Sampler()

num_qubits = 8
num_clbits = 8

#We can have between 1 to 3 winners
num_winners = random.randint(1,3)
winner_list = []
for i in range(num_winners):
    winner_list.append(random.randint(0,(2**num_qubits) - 1))

#Now we have our winners ready.
#We will not print them until the end when we verify our outputs.

#Let's define the classical oracle.
def Classical_Oracle(input):
    if input in winner_list:
        return True
    else:
        return False

#Now create the quantum oracle using it.
#Note that this is happening fully inside the server process as a one time startup
#Technically, we would not be calling the classical oracle
#but rather defining the quantum oracle just like how we defined the classical one.
#Note that on a circuit gate level, the function defined above is also
#a construction based on known answers.

#Let's define the unitary matrix representing the quantum oracle.
outputs = np.fromiter(((-1)**(int(Classical_Oracle(x))) for x in range(2**num_qubits)), dtype=int)
U = np.diag(outputs)
#This unitary matrix flips the phase of the winners.

def Quantum_Oracle(qc):
    gate = UnitaryGate(U,label="QuantumOracle")
    qc.append(gate,qc.qubits)

Step 2: Define the Grover transformation using oracle and reflection along the diffuser. Everything from here on would be expected to be on the client and each call to oracle would be like pinging the server.

In [3]:
def mcz(qc):
    qc.h(num_qubits-1)
    qc.mcx(list(range(num_qubits-1)),num_qubits-1)
    qc.h(num_qubits-1)


def diffuser(qc):
    qc.h(qc.qubits)
    qc.x(qc.qubits)
    mcz(qc)
    qc.x(qc.qubits)
    qc.h(qc.qubits)

Step 3: Now define the full circuit as iterations.

In [4]:
def GroverLayer(qc,num_layers):
    for i in range(num_layers):
        Quantum_Oracle(qc)
        diffuser(qc)

Step 4: Now run with incrementally increasing layers

In [5]:
total_oracle_calls = 0
for iters in range(10):
    max_num_layers = 1
    num_tries = 0
    oracle_calls = 0
    while num_tries < sqrt(2**num_qubits):
        num_layers = random.randint(1,max_num_layers)
        qc = QuantumCircuit(num_qubits,num_clbits)
        qc.h(qc.qubits)
        GroverLayer(qc,num_layers)
        qc.measure(qc.qubits,qc.clbits)
        job = sampler.run(qc,shots=1)
        result = job.result()
        qdists = result.quasi_dists[0]
        output = list(qdists.keys())[0]
        if Classical_Oracle(output):
            oracle_calls += 1
            print('Found matching element ',output,' oracle calls ',oracle_calls)
            print('Winners ',winner_list)
            break
        else:
            oracle_calls += 1
        num_tries += 1
        max_num_layers = ceil(5*max_num_layers/4)
    if num_tries >= sqrt(2**num_qubits):
        print('No Matching result exist oracle calls ',oracle_calls)
        print('Winners ',winner_list)
    total_oracle_calls += oracle_calls
print('Average Oracle calls over 10 iterations ',total_oracle_calls/10)

Found matching element  57  oracle calls  4
Winners  [57, 82]
Found matching element  82  oracle calls  2
Winners  [57, 82]
Found matching element  82  oracle calls  2
Winners  [57, 82]
Found matching element  57  oracle calls  4
Winners  [57, 82]
Found matching element  82  oracle calls  6
Winners  [57, 82]
Found matching element  57  oracle calls  6
Winners  [57, 82]
Found matching element  82  oracle calls  5
Winners  [57, 82]
Found matching element  57  oracle calls  6
Winners  [57, 82]
Found matching element  57  oracle calls  8
Winners  [57, 82]
Found matching element  82  oracle calls  1
Winners  [57, 82]
Average Oracle calls over 10 iterations  4.4


In [6]:
#Classical Comparison case
total_oracle_calls = 0
for iters in range(10):
    oracle_calls = 0
    for output in range(1,2**num_clbits):
        if Classical_Oracle(output):
            oracle_calls += 1
            print('Found matching element ',output,' oracle calls ',oracle_calls)
            print('Winners ',winner_list)
            break
        else:
            oracle_calls += 1
    total_oracle_calls += oracle_calls
print('Average Oracle calls over 10 iterations ',total_oracle_calls/10)

Found matching element  57  oracle calls  57
Winners  [57, 82]
Found matching element  57  oracle calls  57
Winners  [57, 82]
Found matching element  57  oracle calls  57
Winners  [57, 82]
Found matching element  57  oracle calls  57
Winners  [57, 82]
Found matching element  57  oracle calls  57
Winners  [57, 82]
Found matching element  57  oracle calls  57
Winners  [57, 82]
Found matching element  57  oracle calls  57
Winners  [57, 82]
Found matching element  57  oracle calls  57
Winners  [57, 82]
Found matching element  57  oracle calls  57
Winners  [57, 82]
Found matching element  57  oracle calls  57
Winners  [57, 82]
Average Oracle calls over 10 iterations  57.0


Step 5: We can see that when we use the values generated by the quantum circuit and verify them using the classical circuit, we get to one of the winners in the winner list in much lower number of oracle calls as compared to when we use the blind approach of testing every input possible one by one. Now let's try running the quantum circuit on real hardware. To avoid accessing IBM hardware repeatedly, we will use multiple shots. We will use 4 qubits here as Grover cicuits tend to be very deep and can take a lot of time to transpile for high number of qubits. First let's redefine the problem with 4 bits.

In [34]:
num_qubits = 4
num_clbits = 4

#We can have between 1 to 3 winners
num_winners = random.randint(1,3)
winner_list = []
for i in range(num_winners):
    winner_list.append(random.randint(0,(2**num_qubits) - 1))

#Let's define the classical oracle.
def Classical_Oracle(input):
    if input in winner_list:
        return True
    else:
        return False

#Let's define the unitary matrix representing the quantum oracle.
outputs = np.fromiter(((-1)**(int(Classical_Oracle(x))) for x in range(2**num_qubits)), dtype=int)
U = np.diag(outputs)

def Quantum_Oracle(qc):
    gate = UnitaryGate(U,label="QuantumOracle")
    qc.append(gate,qc.qubits)

#Classical Comparison case
total_oracle_calls = 0
for iters in range(10):
    oracle_calls = 0
    for output in range(1,2**num_clbits):
        if Classical_Oracle(output):
            oracle_calls += 1
            print('Found matching element ',output,' oracle calls ',oracle_calls)
            print('Winners ',winner_list)
            break
        else:
            oracle_calls += 1
    total_oracle_calls += oracle_calls
print('Average Oracle calls over 10 iterations ',total_oracle_calls/10)

Found matching element  8  oracle calls  8
Winners  [8, 15, 9]
Found matching element  8  oracle calls  8
Winners  [8, 15, 9]
Found matching element  8  oracle calls  8
Winners  [8, 15, 9]
Found matching element  8  oracle calls  8
Winners  [8, 15, 9]
Found matching element  8  oracle calls  8
Winners  [8, 15, 9]
Found matching element  8  oracle calls  8
Winners  [8, 15, 9]
Found matching element  8  oracle calls  8
Winners  [8, 15, 9]
Found matching element  8  oracle calls  8
Winners  [8, 15, 9]
Found matching element  8  oracle calls  8
Winners  [8, 15, 9]
Found matching element  8  oracle calls  8
Winners  [8, 15, 9]
Average Oracle calls over 10 iterations  8.0


In [11]:
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler, QiskitRuntimeService
service = QiskitRuntimeService(name='simaccount',filename=r'C:\Users\sarth\Documents\Qiskit\Accounts\simaccount.json')

In [12]:
#print(service.active_account())
print(service.backends())
#print(service.saved_accounts(filename=r'C:\Users\sarth\Documents\Qiskit\Accounts\simaccount.json'))
print(service.least_busy(operational=True, simulator=False, min_num_qubits=num_qubits))
import qiskit
print(qiskit.__version__)

[<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_kyiv')>, <IBMBackend('ibm_sherbrooke')>]
<IBMBackend('ibm_kyiv')>
1.1.1


In [39]:
from qiskit_aer import AerSimulator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler, QiskitRuntimeService
from math import sqrt,ceil,floor
from collections import Counter

backend = service.least_busy(operational=True, simulator=False, min_num_qubits=num_qubits)
aer_sim = AerSimulator()
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
real_sampler = Sampler(mode=backend)
total_oracle_calls = 0
total_quantum_shots = 0
max_num_layers = 1
num_shots = sqrt(2**num_qubits)
pub = []
for i in range(floor(num_shots)):
    num_layers = random.randint(1,max_num_layers)
    qc = QuantumCircuit(num_qubits,num_clbits)
    qc.h(qc.qubits)
    GroverLayer(qc,num_layers)
    qc.measure(qc.qubits,qc.clbits)
    print(qc)
    isa_qc = pm.run(qc)
    print(i+1,' out of ',floor(num_shots),' transpiled')
    pub.append(((isa_qc, None, 10)))
    max_num_layers = ceil(5*max_num_layers/4)
total_quantum_shots += floor(num_shots*10)

     ┌───┐┌────────────────┐┌───┐┌───┐          ┌───┐┌───┐     ┌─┐         
q_0: ┤ H ├┤0               ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────┤M├─────────
     ├───┤│                │├───┤├───┤       │  ├───┤├───┤     └╥┘┌─┐      
q_1: ┤ H ├┤1               ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────╫─┤M├──────
     ├───┤│  QuantumOracle │├───┤├───┤       │  ├───┤├───┤      ║ └╥┘┌─┐   
q_2: ┤ H ├┤2               ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────╫──╫─┤M├───
     ├───┤│                │├───┤├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐ ║  ║ └╥┘┌─┐
q_3: ┤ H ├┤3               ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├─╫──╫──╫─┤M├
     └───┘└────────────────┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘ ║  ║  ║ └╥┘
c: 4/═══════════════════════════════════════════════════════════╩══╩══╩══╩═
                                                                0  1  2  3 
1  out of  4  transpiled
     ┌───┐┌────────────────┐┌───┐┌───┐          ┌───┐┌───┐     »
q_0: ┤ H ├┤0               ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────»
     ├───

In [40]:
job = real_sampler.run(pub,shots=None)

In [41]:
print(job.status())

DONE


In [42]:
total_oracle_calls = 0
result = job.result()
for iters in range(10):
    oracle_calls = 0
    num_tries = 0
    out = []
    for j in range(len(result)):
        out.append(result[j].data.c.array[iters][0])
    sorted_out = Counter(out).most_common()
    print(sorted_out)
    size_out = len(sorted_out)
    while num_tries < size_out:
        output = sorted_out[num_tries][0]
        if Classical_Oracle(output):
            oracle_calls += 1
            print('Found matching element ',output,' oracle calls ',oracle_calls)
            print('Winners ',winner_list)
            break
        else:
            oracle_calls += 1
        num_tries += 1
    if num_tries >= size_out:
        print('No Matching result exist oracle calls ',oracle_calls)
        print('Winners ',winner_list)
    total_oracle_calls += oracle_calls
print('Average Oracle calls over 10 iterations ',total_oracle_calls/10,' Average shots on quantum circuit ',total_quantum_shots/10)

[(15, 2), (2, 1), (9, 1)]
Found matching element  15  oracle calls  1
Winners  [8, 15, 9]
[(9, 2), (3, 1), (0, 1)]
Found matching element  9  oracle calls  1
Winners  [8, 15, 9]
[(0, 1), (8, 1), (11, 1), (15, 1)]
Found matching element  8  oracle calls  2
Winners  [8, 15, 9]
[(8, 1), (13, 1), (9, 1), (10, 1)]
Found matching element  8  oracle calls  1
Winners  [8, 15, 9]
[(14, 1), (1, 1), (6, 1), (0, 1)]
No Matching result exist oracle calls  4
Winners  [8, 15, 9]
[(4, 2), (7, 1), (5, 1)]
No Matching result exist oracle calls  3
Winners  [8, 15, 9]
[(14, 2), (15, 1), (9, 1)]
Found matching element  15  oracle calls  2
Winners  [8, 15, 9]
[(3, 1), (12, 1), (5, 1), (8, 1)]
Found matching element  8  oracle calls  4
Winners  [8, 15, 9]
[(14, 1), (7, 1), (0, 1), (13, 1)]
No Matching result exist oracle calls  4
Winners  [8, 15, 9]
[(12, 1), (9, 1), (5, 1), (0, 1)]
Found matching element  9  oracle calls  2
Winners  [8, 15, 9]
Average Oracle calls over 10 iterations  2.4  Average shots on q

Out of the 10 test cases, 3 cases found result in 1 oracle call, 3 cases found result in 2 oracle calls, 1 case found result in 3 oracle calls, 1 case found result in 4 oracle calls, and 2 cases were not able to find results with the current number of shots on the quantum circuit. This shows that the noise in real quantum circuits is still a challenge but when it does work, it is able to obtain the result in a much lower number of oracle calls than 8 which is the classical case.